In [3]:
import pandas as pd
import numpy as np
from pathlib import Path

BASE_DIR = Path.cwd().parents[0] if Path.cwd().name == "notebooks" else Path.cwd()
DATA_DIR = BASE_DIR / "data" / "processed_refined"
df = pd.read_csv(DATA_DIR / "brasileirao_refinado.csv")

print("Antes do tratamento:")
print(df.isna().sum().sort_values(ascending=False).head(10))

# --- Preenchimentos avançados ---

# 1. Técnicos e árbitros: preenche com 'Desconhecido'
df["tecnico_mandante"] = df["tecnico_mandante"].fillna("Desconhecido")
df["tecnico_visitante"] = df["tecnico_visitante"].fillna("Desconhecido")
df["arbitro"] = df["arbitro"].fillna("Não informado")

# 2. Públicos: substitui NaN por média móvel do campeonato
for col in ["publico", "publico_max"]:
    df[col] = df.groupby("ano_campeonato")[col].transform(lambda x: x.fillna(x.median()))
    df[col] = df[col].fillna(df[col].median())

# 3. Estatísticas zeradas: se todas forem 0, substitui por média da temporada
cols_estat = [
    "chutes_mandante", "chutes_visitante",
    "chutes_fora_mandante", "chutes_fora_visitante",
    "escanteios_mandante", "escanteios_visitante",
    "faltas_mandante", "faltas_visitante",
    "defesas_mandante", "defesas_visitante"
]

for col in cols_estat:
    def substituir_zero_por_mediana(x):
        if (x > 0).any():
            mediana = np.nanmedian(x[x > 0])
            return x.replace(0, mediana)
        else:
            return x  # deixa 0 se não há base de dados
    df[col] = df.groupby("ano_campeonato")[col].transform(substituir_zero_por_mediana)

print("\nDepois do tratamento:")
print(df.isna().sum().sort_values(ascending=False).head(10))


Antes do tratamento:
tecnico_mandante                  2154
tecnico_visitante                 2154
idade_media_titular_visitante     2100
idade_media_titular_mandante      2100
valor_equipe_titular_visitante    2098
valor_equipe_titular_mandante     2098
publico_max                       2090
gols_1_tempo_mandante             1721
gols_1_tempo_visitante            1721
colocacao_mandante                1710
dtype: int64

Depois do tratamento:
idade_media_titular_visitante     2100
idade_media_titular_mandante      2100
valor_equipe_titular_visitante    2098
valor_equipe_titular_mandante     2098
gols_1_tempo_visitante            1721
gols_1_tempo_mandante             1721
colocacao_mandante                1710
colocacao_visitante               1710
estadio                             11
gols_mandante                        1
dtype: int64


/Users/fabrica/Documents/ia-futebol-brasil/.venv/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/fabrica/Documents/ia-futebol-brasil/.venv/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/fabrica/Documents/ia-futebol-brasil/.venv/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/fabrica/Documents/ia-futebol-brasil/.venv/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/fabrica/Documents/ia-futebol-brasil/.venv/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.

In [4]:
df["gols_mandante"] = df["gols_mandante"].fillna(0)
df["gols_1_tempo_mandante"] = df["gols_1_tempo_mandante"].fillna(0)
df["gols_1_tempo_visitante"] = df["gols_1_tempo_visitante"].fillna(0)
df["estadio"] = df["estadio"].fillna("Estádio não informado")

print("Após ajustes finais:")
print(df.isna().sum().sort_values(ascending=False).head(10))


Após ajustes finais:
idade_media_titular_visitante     2100
idade_media_titular_mandante      2100
valor_equipe_titular_mandante     2098
valor_equipe_titular_visitante    2098
colocacao_visitante               1710
colocacao_mandante                1710
resultado                            1
gols_visitante                       1
chutes_bola_parada_visitante         0
defesas_mandante                     0
dtype: int64


In [5]:
OUTPUT_DIR = DATA_DIR.parent / "final"
OUTPUT_DIR.mkdir(exist_ok=True)
df.to_csv(OUTPUT_DIR / "brasileirao_final.csv", index=False)
print(f"Base final salva em: {OUTPUT_DIR / 'brasileirao_final.csv'}")


Base final salva em: /Users/fabrica/Documents/ia-futebol-brasil/data/final/brasileirao_final.csv
